In [1]:
#installing extra libraries on prod
!pip install google-api-core==2.7.1 google-api-python-client==2.42.0 google-auth==2.6.6
!pip install --upgrade google-api-python-client oauth2client
!pip install zeno-etl-libs==1.0.40
!pip install google-cloud-storage==1.43.0

  Using cached google_api_python_client-2.42.0-py2.py3-none-any.whl (8.3 MB)
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.51.0
    Uninstalling google-api-python-client-2.51.0:
      Successfully uninstalled google-api-python-client-2.51.0


  Using cached google_api_python_client-2.51.0-py2.py3-none-any.whl (8.6 MB)
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.42.0
    Uninstalling google-api-python-client-2.42.0:
      Successfully uninstalled google-api-python-client-2.42.0


     -------------------------------------- 431.0/431.0 kB 4.5 MB/s eta 0:00:00


In [2]:
"""
Fetching Playstore reviews on daily basis
Author : neha.karekar@zeno.health
"""

'\nFetching Playstore reviews on daily basis\nAuthor : neha.karekar@zeno.health\n'

In [3]:
import argparse
import sys
import os
import pandas as pd
import dateutil
import datetime
from dateutil.tz import gettz
import numpy as np

In [4]:
sys.path.append('../../../..')

In [5]:
from zeno_etl_libs.logger import get_logger
from zeno_etl_libs.db.db import DB
from zeno_etl_libs.helper.aws.s3 import S3
from zeno_etl_libs.helper import helper
from zeno_etl_libs.helper.google.playstore.playstore import Reviews

## Pass Params


In [6]:
env = "dev"
full_run = 0

In [7]:
os.environ['env'] = env
logger = get_logger()
logger.info(f"full_run: {full_run}")


2022-06-23 10:55:19,964 - root - INFO - full_run: 1


In [8]:
rs_db = DB(read_only=False)
rs_db.open_connection()
s3 = S3()

In [9]:
schema = 'prod2-generico'
table_name = 'ecomm-playstore-reviews'
table_info = helper.get_table_info(db=rs_db, table_name=table_name, schema=schema)

In [10]:
# max of data
playstore_q = """
select
            max("review-created-at") max_exp
        from
            "prod2-generico"."ecomm-playstore-reviews" 
        """
rs_db.execute(playstore_q, params=None)
max_exp_date: pd.DataFrame = rs_db.cursor.fetch_dataframe()
max_exp_date['max_exp'].fillna(np.nan, inplace=True)
print(max_exp_date.info())
max_exp_date = max_exp_date['max_exp'].to_string(index=False)
print(max_exp_date)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   max_exp  1 non-null      datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 136.0 bytes
None
2022-06-17 15:28:31


In [11]:
# Read from gsheet
reviews = Reviews()
reviews_list = reviews.get()
# reviews_list
reviews_list_new = []
for r in reviews_list:
    for c in r['comments']:
        rc = {}
        rc.update({"reviewId": r['reviewId'], "authorName": r['authorName']})
        rc.update(c)
        reviews_list_new.append(rc)
df = pd.json_normalize(reviews_list_new)
df = df[['reviewId', 'authorName', 'userComment.text', 'userComment.starRating', 'userComment.reviewerLanguage',
          'userComment.lastModified.seconds']]
df = df[(df['userComment.text'].isnull() == False)]

2022-06-23 10:55:23,277 - googleapiclient.discovery_cache - INFO - file_cache is only supported with oauth2client<4.0.0
2022-06-23 10:55:23,286 - oauth2client.transport - INFO - Attempting refresh to obtain initial access_token
2022-06-23 10:55:23,317 - oauth2client.client - INFO - Refreshing access_token
b'\tnice aap \xf0\x9f\x91\x8c\xf0\x9f\x91\x8c'
b'\tBakwaas App and service , everytime they are out of stock despite taking orders on App'
b'\tPlenty of alternative medicines at very affordable price and quickest door delivery. Love this. Getting Coins and adjusting with order is also appreciated..... Only need to modify/rectify during checkout.'
b'\tMany medicine remain out of stock at time of delivery and no care to restock at earliest. Poor management'
b'\tVery bad service not giving on time always late I have try 4 time all time late delivery'
b'\tgood medicine with money saving'
b'\tReally useful app. It saves a tonne of money and is easy to use. The Customer support responds wit

In [12]:
dict = {'reviewId': 'review-id',
        'userComment.text': 'review',
        'authorName': 'author-name',
        'userComment.starRating': 'star-rating',
        'userComment.reviewerLanguage': 'reviewer-lang'}
df.rename(columns=dict, inplace=True)
print()
df['review-created-at'] = df['userComment.lastModified.seconds'].apply(
    lambda x: datetime.datetime.fromtimestamp(int(x)).strftime('%Y-%m-%d %H:%M:%S'))
df=df.drop(['userComment.lastModified.seconds'],axis=1)

In [13]:
# params
if full_run or max_exp_date == 'NaN':
    start = '2017-05-13'
else:
    start = max_exp_date
start = dateutil.parser.parse(start)

In [14]:
df['review-created-at'] = df['review-created-at'] .apply(pd.to_datetime, errors='coerce')
df = df[(df['review-created-at'] > start)]

In [15]:
# etl
df['created-at'] = datetime.datetime.now(tz=gettz('Asia/Kolkata')).strftime('%Y-%m-%d %H:%M:%S')
df['updated-at'] = datetime.datetime.now(tz=gettz('Asia/Kolkata')).strftime('%Y-%m-%d %H:%M:%S')
df['created-by'] = 'etl-automation'
df['updated-by'] = 'etl-automation'
df.columns = [c.replace('_', '-') for c in df.columns]

In [16]:
if isinstance(table_info, type(None)):
    raise Exception(f"table: {table_name} do not exist, create the table first")
else:
    logger.info(f"Table:{table_name} exists")

2022-06-23 10:55:39,612 - root - INFO - Table:ecomm-playstore-reviews exists


In [17]:
truncate_query = f''' DELETE FROM "{schema}"."{table_name}" where "review-created-at" >'{start}' '''
print(truncate_query)
rs_db.execute(truncate_query)

 DELETE FROM "prod2-generico"."ecomm-playstore-reviews" where "review-created-at" >'2017-05-13 00:00:00' 


In [18]:
s3.write_df_to_db(df=df[table_info['column_name']], table_name=table_name, db=rs_db,
                      schema=schema)

S3 object(uri: s3://aws-glue-temporary-921939243643-ap-south-1/temp_1655961949791.csv) delete response: {'ResponseMetadata': {'RequestId': '9CVT8G0EAN6BNVX0', 'HostId': 'xk+MzVbdwZg4EWIjBtGBObnOExi0kDsso7yvSvMKmR8z29sd4LWCvMU+SwqWOSzqydpM5L3LaTw=', 'HTTPStatusCode': 204, 'HTTPHeaders': {'x-amz-id-2': 'xk+MzVbdwZg4EWIjBtGBObnOExi0kDsso7yvSvMKmR8z29sd4LWCvMU+SwqWOSzqydpM5L3LaTw=', 'x-amz-request-id': '9CVT8G0EAN6BNVX0', 'date': 'Thu, 23 Jun 2022 05:26:02 GMT', 'server': 'AmazonS3'}, 'RetryAttempts': 0}}
S3 object(uri: s3://aws-glue-temporary-921939243643-ap-south-1/temp_1655961949791.csv) delete response: {'ResponseMetadata': {'RequestId': '9CVJB4WVG0CK02J0', 'HostId': '5HhUTM1mH6NhEfiS44ZfggPv0E8zQXC0BwoDYP4EUNrExVRA0NpdS5boG9IhrDHrka0p07k9yxk=', 'HTTPStatusCode': 204, 'HTTPHeaders': {'x-amz-id-2': '5HhUTM1mH6NhEfiS44ZfggPv0E8zQXC0BwoDYP4EUNrExVRA0NpdS5boG9IhrDHrka0p07k9yxk=', 'x-amz-request-id': '9CVJB4WVG0CK02J0', 'date': 'Thu, 23 Jun 2022 05:26:02 GMT', 'server': 'AmazonS3'}, 'RetryA

In [19]:
rs_db.close_connection()

Redshift DB connection closed successfully.
